In [1]:
import os
import numpy as np
from math import fabs

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def robust_min(img, p=5):
    return np.percentile(img.ravel(), p)


def robust_max(img, p=95):
    return np.percentile(img.ravel(), p)


def normalize(img, m=10, M=90):
    return np.clip((img - robust_min(img, m)) / (robust_max(img, M) - robust_min(img, m)), 0.0, 1.0)


def first_element_greater_than(values, req_value):
    """Returns the pair (i, values[i]) such that i is the minimum value that satisfies values[i] >= req_value.
    Returns (-1, None) if there is no such i.
    Note: this function assumes that values is a sorted array!"""
    i = np.searchsorted(values, req_value)
    val = values[i] if i < len(values) else None
    return (i, val)


def last_element_less_than(values, req_value):
    """Returns the pair (i, values[i]) such that i is the maximum value that satisfies values[i] <= req_value.
    Returns (-1, None) if there is no such i.
    Note: this function assumes that values is a sorted array!"""
    i = np.searchsorted(values, req_value, side='right') - 1
    val = values[i] if i >= 0 else None
    return (i, val)


def closest_element_to(values, req_value):
    """Returns the tuple (i, values[i], diff) such that i is the closest value to req_value,
    and diff = |values(i) - req_value|
    Note: this function assumes that values is a sorted array!"""
    assert(len(values) > 0)

    i = np.searchsorted(values, req_value, side='left')
    if i > 0 and (i == len(values) or fabs(req_value - values[i - 1]) < fabs(req_value - values[i])):
        idx = i - 1
        val = values[i - 1]
    else:
        idx = i
        val = values[i]

    diff = fabs(val - req_value)
    return (idx, val, diff)


In [8]:
data = EventDataset("C:\\Users\\wasse\\rpg_e2depth\\data\\", "test_sequence_00_town10\\events\\data\\", start_time=0, stop_time=0, transform=None, normalize=True)

C:\Users\wasse\AppData\Local\Temp\ipykernel_21684\14396457.py:20: DeprecationWarning: `alltrue` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `all` instead.
  self.read_timestamps()


In [15]:
VoxelGridDataset("C:\\Users\\wasse\\rpg_e2depth\\data\\", "test_sequence_00_town10\\events\\data\\", start_time=0, stop_time=0, transform=None, normalize=True)

C:\Users\wasse\AppData\Local\Temp\ipykernel_21684\14396457.py:20: DeprecationWarning: `alltrue` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `all` instead.
  self.read_timestamps()


In [13]:
class VoxelGridDataset(EventDataset):
    """Load an event folder containing event tensors encoded with the VoxelGrid format."""

    def parse_event_folder(self):
        """Check that the passed directory has the following form:

        ├── event_folder
        |   ├── timestamps.txt
        |   ├── event_tensor_0000000000.npy
        |   ├── ...
        |   ├── event_tensor_<N>.npy
        """
        self.num_bins = None

    def num_channels(self):
        return self.num_bins

    def __getitem__(self, i, transform_seed=None):
        assert(i >= 0)
        assert(i < self.length)

        if transform_seed is None:
            transform_seed = random.randint(0, 2**32)

        # event_tensor will be a [num_bins x H x W] floating point array
        event_tensor = np.load(join(self.event_folder, 'event_tensor_{:010d}.npy'.format(self.first_valid_idx + i)))
        if self.normalize:
            # normalize the event tensor (voxel grid) in such a way that the mean and stddev of the nonzero values
            # in the tensor are equal to (0.0, 1.0)
            mask = np.nonzero(event_tensor)
            if mask[0].size > 0:
                mean, stddev = event_tensor[mask].mean(), event_tensor[mask].std()
                if stddev > 0:
                    event_tensor[mask] = (event_tensor[mask] - mean) / stddev

        self.num_bins = event_tensor.shape[0]

        events = torch.from_numpy(event_tensor)  # [C x H x W]
        if self.transform:
            random.seed(transform_seed)
            events = self.transform(events)

        return {'events': events}  # [num_bins x H x W] tensor


In [16]:
import dv_processing as dv
import numpy as np
import torch
from os.path import join
from datetime import timedelta
import cv2 as cv

# Event filtering classes
class AdaptiveThresholdFilter:
    def __init__(self, resolution, adaptation_rate=0.01, decay_rate=0.1):
        self.resolution = resolution
        self.adaptation_rate = adaptation_rate
        self.decay_rate = decay_rate
        self.threshold_map = np.zeros(resolution, dtype=np.float32)

    def apply(self, events):
        filtered_events = []
        for event in events:
            x, y = int(event[0]), int(event[1])
            if 0 <= x < self.resolution[0] and 0 <= y < self.resolution[1]:
                self.threshold_map[x, y] += self.adaptation_rate
                if abs(event[3]) > self.threshold_map[x, y]:
                    filtered_events.append(event)
                self.threshold_map[x, y] *= (1 - self.decay_rate)
        return np.array(filtered_events)

def density_filter(events, resolution, min_density=10, max_density=100):
    width, height = resolution
    grid = np.zeros((width, height), dtype=np.int32)
    for event in events:
        x, y = int(event[0]), int(event[1])
        if 0 <= x < width and 0 <= y < height:
            grid[x, y] += 1
    return np.array([event for event in events if min_density <= grid[int(event[0]), int(event[1])] <= max_density])

# Dataset class
class NPYEventDataset:
    def __init__(self, base_folder, event_folder, resolution=(640, 480)):
        self.base_folder = base_folder
        self.event_folder = join(self.base_folder, event_folder)
        self.resolution = resolution
        self.files = self.load_event_files()

    def load_event_files(self):
        import glob
        return sorted(glob.glob(join(self.event_folder, 'event_tensor_*.npy')))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        data = np.load(self.files[idx])  # Load raw numpy event data
        return data.reshape(-1, 4)  # Expecting [x, y, timestamp, polarity]

def process_events(dataset, filter_type="adaptive"):
    total_original_events = 0
    total_filtered_events = 0

    adaptive_filter = AdaptiveThresholdFilter(resolution=(640, 480))

    for i in range(len(dataset)):
        events = dataset[i]
        total_original_events += len(events)

        if filter_type == "density":
            filtered_events = density_filter(events, resolution=(640, 480))
        elif filter_type == "adaptive":
            filtered_events = adaptive_filter.apply(events)
        else:
            raise ValueError("Unknown filter type")

        total_filtered_events += len(filtered_events)

        # Save filtered events
        np.savetxt(f'filtered_events_{i}.txt', filtered_events, fmt='%d')

        print(f"File {i}: Original events = {len(events)}, Filtered events = {len(filtered_events)}")

    print(f"Total original events: {total_original_events}")
    print(f"Total filtered events: {total_filtered_events}")

# Usage

base_folder = "C:\\Users\\wasse\\rpg_e2depth\\data\\"
event_folder = "test_sequence_00_town10\\events\\data\\"
dataset = NPYEventDataset(base_folder, event_folder)
process_events(dataset, filter_type="adaptive")


Total original events: 0
Total filtered events: 0


In [20]:
np.load("C:\\Users\\wasse\\rpg_e2depth\\data\\test_sequence_00_town10\\events\\data\\events_0000000003.npy").shape

(6940, 4)

In [36]:
import dv_processing as dv
import numpy as np
import cv2 as cv
from datetime import timedelta
import os

class AdaptiveThresholdFilter:
    def __init__(self, resolution, adaptation_rate=0.1, decay_rate=0.05):
        self.resolution = resolution
        self.adaptation_rate = adaptation_rate
        self.decay_rate = decay_rate
        self.threshold_map = np.zeros(resolution, dtype=np.float32)

    def apply(self, events):
        filtered_events = []
        for event in events:
            x, y = int(event.x()), int(event.y())  # x, y coordinates
            if 0 <= x < self.resolution[0] and 0 <= y < self.resolution[1]:
                self.threshold_map[x, y] += self.adaptation_rate
                #print(f"Threshold for ({x},{y}): {self.threshold_map[x, y]}")  # Debugging
                if abs(event.polarity()) > self.threshold_map[x, y]:  # Polarity thresholding
                    filtered_events.append(event)
                self.threshold_map[x, y] *= (1 - self.decay_rate)
        return filtered_events


def density_filter(events, resolution, min_density=10, max_density=100):
    width, height = resolution
    grid = np.zeros((width, height), dtype=np.int32)
    for event in events:
        x, y = int(event.x()), int(event.y())  # x, y coordinates
        if 0 <= x < width and 0 <= y < height:
            grid[x, y] += 1
    #print(f"Grid density: {grid}")  # Debugging grid density
    return [event for event in events if min_density <= grid[int(event.x()), int(event.y())] <= max_density]


def save_events(events, output_file):
    with open(output_file, "w") as f:
        for event in events:
            f.write(f"{event.timestamp()},{event.x()},{event.y()},{event.polarity()}\n")

def load_event_data(npy_folder):
    event_files = sorted([f for f in os.listdir(npy_folder) if f.endswith('.npy')])
    all_events = dv.EventStore()
    for file in event_files:
        file_path = os.path.join(npy_folder, file)
        events = np.load(file_path)  # Load events (timestamp, x, y, polarity)
        
        for event in events:
            timestamp, x, y, polarity = event
            # Create dv.Event and add to EventStore
            new_event = dv.Event(int(timestamp), int(x), int(y), bool(polarity))
            all_events.push_back(new_event)
    
    return all_events

# Load event data
npy_folder = "C:\\Users\\wasse\\rpg_e2depth\\data\\test_sequence_00_town10\\events\\data\\"
all_events = load_event_data(npy_folder)

# Initialize filter
adaptive_filter = AdaptiveThresholdFilter(resolution=(640, 480))
filter_type = "density" 

# Apply filtering
if filter_type == "density":
    filtered_events = density_filter(all_events, resolution=(640, 480))
elif filter_type == "adaptive":
    filtered_events = adaptive_filter.apply(all_events)

# Save filtered events
save_events(filtered_events, "filtered_events.txt")
    
print(f"Total original events: {len(all_events)}")
print(f"Total filtered events: {len(filtered_events)}")

Total original events: 4422840
Total filtered events: 3667420


In [22]:
import dv_processing as dv
import numpy as np
import cv2 as cv
from datetime import timedelta
import os

class AdaptiveThresholdFilter:
    def __init__(self, resolution, adaptation_rate=0.01, decay_rate=0.1):
        self.resolution = resolution
        self.adaptation_rate = adaptation_rate
        self.decay_rate = decay_rate
        self.threshold_map = np.zeros(resolution, dtype=np.float32)

    def apply(self, events):
        filtered_events = []
        for event in events:
            x, y = int(event[1]), int(event[2])  # x, y coordinates
            if 0 <= x < self.resolution[0] and 0 <= y < self.resolution[1]:
                self.threshold_map[x, y] += self.adaptation_rate
                if abs(event[3]) > self.threshold_map[x, y]:  # Polarity thresholding
                    filtered_events.append(event)
                self.threshold_map[x, y] *= (1 - self.decay_rate)
        return filtered_events

def density_filter(events, resolution, min_density=10, max_density=100):
    width, height = resolution
    grid = np.zeros((width, height), dtype=np.int32)
    for event in events:
        x, y = int(event[1]), int(event[2])  # x, y coordinates
        if 0 <= x < width and 0 <= y < height:
            grid[x, y] += 1
    return [event for event in events if min_density <= grid[int(event[1]), int(event[2])] <= max_density]

def save_events(events, output_file):
    np.savetxt(output_file, events, fmt="%.6f,%d,%d,%d", delimiter=",")

def load_event_data(npy_folder):
    event_files = sorted([f for f in os.listdir(npy_folder) if f.endswith('.npy')])
    all_events = []
    for file in event_files:
        file_path = os.path.join(npy_folder, file)
        events = np.load(file_path)  # Load events (timestamp, x, y, polarity)
        all_events.extend(events)
    return np.array(all_events)

# Load event data
npy_folder = "C:\\Users\\wasse\\rpg_e2depth\\data\\test_sequence_00_town10\\events\\data\\"
all_events = load_event_data(npy_folder)

# Initialize filter
adaptive_filter = AdaptiveThresholdFilter(resolution=(640, 480))
filter_type = "adaptive"  # Change to "density" for density filtering

# Apply filtering
if filter_type == "density":
    filtered_events = density_filter(all_events, resolution=(640, 480))
elif filter_type == "adaptive":
    filtered_events = adaptive_filter.apply(all_events)

# Save filtered events
save_events(filtered_events, "filtered_events.txt")

print(f"Total original events: {len(all_events)}")
print(f"Total filtered events: {len(filtered_events)}")

Total original events: 4422840
Total filtered events: 4422840


In [ ]:
import dv_processing as dv
import numpy as np
import cv2 as cv
from datetime import timedelta
import os

class AdaptiveThresholdFilter:
    def __init__(self, resolution, adaptation_rate=0.01, decay_rate=0.1):
        self.resolution = resolution
        self.adaptation_rate = adaptation_rate
        self.decay_rate = decay_rate
        self.threshold_map = np.zeros(resolution, dtype=np.float32)

    def apply(self, events):
        filtered_events = []
        for event in events:
            x, y = int(event.x()), int(event.y())
            if 0 <= x < self.resolution[0] and 0 <= y < self.resolution[1]:
                self.threshold_map[x, y] += self.adaptation_rate
                if abs(event.polarity()) > self.threshold_map[x, y]:
                    filtered_events.append(event)
                self.threshold_map[x, y] *= (1 - self.decay_rate)
        return filtered_events

def density_filter(events, resolution, min_density=10, max_density=100):
    width, height = resolution
    grid = np.zeros((width, height), dtype=np.int32)
    for event in events:
        x, y = int(event.x()), int(event.y())
        if 0 <= x < width and 0 <= y < height:
            grid[x, y] += 1
    return [
        event for event in events if min_density <= grid[int(event.x()), int(event.y())] <= max_density
    ]

def save_events(events, output_file):
    with open(output_file, "w") as f:
        for event in events:
            f.write(f"{event.x()},{event.y()},{event.timestamp()},{event.polarity()}\n")

def load_npy_as_eventstore(file_path):
    data = np.load(file_path)
    timestamps, x_coords, y_coords, polarities = data[:, 0], data[:, 1], data[:, 2], data[:, 3]

    event_store = dv.EventStore()
    for t, x, y, p in zip(timestamps, x_coords, y_coords, polarities):
        event_store.push_back(int(t), int(x), int(y), bool(p))
    
    return event_store

def callback(left_events: dv.EventStore, right_events: dv.EventStore):
    global total_original_events, total_filtered_events

    # Count original events
    original_event_count = len(left_events) + len(right_events)
    total_original_events += original_event_count

    # Apply filtering
    if filter_Events == "density":
        filtered_left = density_filter(left_events, resolution=(640, 480))
        filtered_right = density_filter(right_events, resolution=(640, 480))
    elif filter_Events == "adaptive":
        filtered_left = adaptive_filter.apply(left_events)
        filtered_right = adaptive_filter.apply(right_events)

    # Count filtered events
    filtered_event_count = len(filtered_left) + len(filtered_right)
    total_filtered_events += filtered_event_count

    # Save filtered events
    save_events(filtered_left, "filtered_left_events.txt")
    save_events(filtered_right, "filtered_right_events.txt")

    print(f"Original events: {original_event_count}, Filtered events: {filtered_event_count}")

# Paths to npy files
left_npy_file = "/path/to/left_events.npy"
right_npy_file = "/path/to/right_events.npy"

# Initialize filter
adaptive_filter = AdaptiveThresholdFilter(resolution=(640, 480))
filter_Events = "adaptive"

# Load events from npy files
left_events = load_npy_as_eventstore(left_npy_file)
right_events = load_npy_as_eventstore(right_npy_file)

slicer = dv.StereoEventStreamSlicer()
slicer.doEveryTimeInterval(timedelta(milliseconds=33), callback)

# Initialize counters
total_original_events = 0
total_filtered_events = 0

# Process loaded event data
slicer.accept(left_events, right_events)
cv.waitKey(1)

# Print final event counts
print(f"Total original events: {total_original_events}")
print(f"Total filtered events ({filter_Events}): {total_filtered_events}")


In [30]:
import dv_processing as dv
import numpy as np
import os

class AdaptiveThresholdFilter:
    def __init__(self, resolution, adaptation_rate=0.01, decay_rate=0.1):
        self.resolution = resolution
        self.adaptation_rate = adaptation_rate
        self.decay_rate = decay_rate
        self.threshold_map = np.zeros(resolution, dtype=np.float32)

    def apply(self, events):
        filtered_events = []
        for event in events:
            x, y = event.x, event.y  # x, y coordinates
            if 0 <= x < self.resolution[0] and 0 <= y < self.resolution[1]:
                self.threshold_map[x, y] += self.adaptation_rate
                if abs(event.polarity) > self.threshold_map[x, y]:  # Polarity thresholding
                    filtered_events.append(event)
                self.threshold_map[x, y] *= (1 - self.decay_rate)
        return filtered_events

def density_filter(events, resolution, min_density=10, max_density=100):
    width, height = resolution
    grid = np.zeros((width, height), dtype=np.int32)
    for event in events:
        x, y = event.x, event.y  # x, y coordinates
        if 0 <= x < width and 0 <= y < height:
            grid[x, y] += 1
    return [event for event in events if min_density <= grid[event.x, event.y] <= max_density]

def load_event_data(npy_folder):
    event_store = dv.EventStore()
    event_files = sorted([f for f in os.listdir(npy_folder) if f.endswith('.npy')])

    for file in event_files:
        file_path = os.path.join(npy_folder, file)
        events_np = np.load(file_path)  # Load events in [timestamp, x, y, polarity] format

        # Create an event vector
        event_vector = dv.EventPacket.EventVector()

        for event in events_np:
            timestamp, x, y, polarity = map(int, event)
            event_vector.append(dv.Event(timestamp, x, y, polarity))

        # Create an event packet with the event vector
        packet = dv.EventPacket(event_vector)
        
        # Add the packet to the event store
        event_store.add(packet)

    return event_store



# Load event data from folder
npy_folder = "C:\\Users\\wasse\\rpg_e2depth\\data\\test_sequence_00_town10\\events\\data\\"
event_store = load_event_data(npy_folder)

# Convert EventStore to a list of dv.Event objects for processing
all_events = list(event_store)

# Apply filtering
adaptive_filter = AdaptiveThresholdFilter(resolution=(640, 480))
filter_type = "adaptive"  # Change to "density" for density filtering

if filter_type == "density":
    filtered_events = density_filter(all_events, resolution=(640, 480))
elif filter_type == "adaptive":
    filtered_events = adaptive_filter.apply(all_events)

# Save filtered events
filtered_event_store = dv.EventStore()
for event in filtered_events:
    filtered_event_store.add(event)

filtered_event_store.save_to_file("filtered_events.aedat4")

print(f"Total original events: {len(all_events)}")
print(f"Total filtered events: {len(filtered_events)}")


TypeError: add(): incompatible function arguments. The following argument types are supported:
    1. (self: dv_processing.EventStore, other: dv_processing.EventStore) -> None

Invoked with: EventStore containing 0 events within 0µs duration; time range within [0; 0], <dv_processing.EventPacket object at 0x00000189855566F0>

In [25]:
?dv.Event

Docstring:      A structure defining a brightness change event captured by an event camera
Init docstring: __init__(self: dv_processing.Event, timestamp: int, x: int, y: int, polarity: bool) -> None
File:           c:\users\wasse\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages\dv_processing.cp310-win_amd64.pyd
Type:           pybind11_type
Subclasses:     

In [21]:
import numpy as np
import dv_processing as dv
import os

def load_event_data(event_folder):
    npy_files = sorted([f for f in os.listdir(event_folder) if f.endswith('.npy')])
    events = []
    for npy_file in npy_files:
        data = np.load(os.path.join(event_folder, npy_file))  # Expecting shape (x, 4)
        for x, y, timestamp, polarity in data:
            event = dv.Event(x=int(x), y=int(y), timestamp=int(timestamp), polarity=int(polarity))
            events.append(event)
    return dv.EventStore(events)

def load_timestamps(event_folder):
    boundary_timestamps = np.loadtxt(os.path.join(event_folder, 'boundary_timestamps.txt'))
    timesteps = np.loadtxt(os.path.join(event_folder, 'timesteps.txt'))
    return boundary_timestamps, timesteps

# Example usage
event_folder = "C:\\Users\\wasse\\rpg_e2depth\\data\\test_sequence_00_town10\\events\\data\\"
event_store = load_event_data(event_folder)
boundary_timestamps, timesteps = load_timestamps(event_folder)

# Now event_store can be passed to the filtering functions

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. dv_processing.Event(timestamp: int, x: int, y: int, polarity: bool)

Invoked with: kwargs: x=4471596111, y=107, timestamp=0, polarity=-1

In [7]:
from torch.utils.data import Dataset
from os.path import join
import numpy as np
import random
import torch


class EventDataset(Dataset):
    """Loads event tensors from a folder, with different event representations."""
    def __init__(self, base_folder, event_folder, start_time=0, stop_time=0, transform=None, normalize=True):
        self.base_folder = base_folder
        self.event_folder = join(self.base_folder, event_folder)
        self.transform = transform

        self.start_time = start_time
        self.stop_time = stop_time

        self.normalize = normalize

        self.read_timestamps()

        #self.parse_event_folder()

    def read_timestamps(self):
        # Load the timestamps file
        raw_stamps = np.loadtxt(join(self.event_folder, 'timestamps.txt'))

        if raw_stamps.size == 0:
            raise IOError('Dataset is empty')

        if len(raw_stamps.shape) == 1:
            # if timestamps.txt has only one entry, the shape will be (2,) instead of (1, 2). fix that.
            raw_stamps = raw_stamps.reshape((1, 2))

        self.stamps = raw_stamps[:, 1]
        if self.stamps is None:
            raise IOError('Unable to read timestamp file: '.format(join(self.event_folder,
                                                                        'timestamps.txt')))

        # Check that the timestamps are unique and sorted
        assert(np.alltrue(np.diff(self.stamps) > 0)), "timestamps are not unique and monotonically increasing"

        self.initial_stamp = self.stamps[0]
        self.stamps = self.stamps - self.initial_stamp  # offset the timestamps so they start at 0

        # Find the index of the first event tensor whose timestamp >= start_time
        # If there is none, throw an error
        if self.start_time <= 0.0:
            self.first_valid_idx, self.first_stamp = 0, self.stamps[0]
        else:
            self.first_valid_idx, self.first_stamp = first_element_greater_than(self.stamps, self.start_time)
        assert(self.first_stamp is not None)
        # print('First valid index / stamp = {}, {}'.format(self.first_valid_idx, self.first_stamp))

        # Find the index of the last event tensor whose timestamp <= end_time
        # If there is None, throw an error
        if self.stop_time <= 0.0:
            self.last_valid_idx, self.last_stamp = len(self.stamps) - 1, self.stamps[-1]
        else:
            self.last_valid_idx, self.last_stamp = last_element_less_than(self.stamps, self.stop_time)
        assert(self.last_stamp is not None)
        # print('Last valid index / stamp = {}, {}'.format(self.last_valid_idx, self.last_stamp))

        assert(self.first_stamp <= self.last_stamp)

        self.length = self.last_valid_idx - self.first_valid_idx + 1
        assert(self.length > 0)

    def parse_event_folder(self):
        """Parses the event folder to check its validity and read the parameters of the event representation."""
        raise NotImplementedError

    def __len__(self):
        return self.length

    def get_last_stamp(self):
        """Returns the last event timestamp, in seconds."""
        return self.stamps[self.last_valid_idx]

    def num_channels(self):
        """Returns the number of channels of the event tensor."""
        raise NotImplementedError

    def get_index_at(self, i):
        """Returns the index of the ith event tensor"""
        return self.first_valid_idx + i

    def get_stamp_at(self, i):
        """Returns the timestamp of the ith event tensor"""
        return self.stamps[self.get_index_at(i)]

    def __getitem(self, i):
        """Returns a C x H x W event tensor for the ith element in the dataset."""
        raise NotImplementedError